<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/usage_example(colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt

fatal: destination path 'c620' already exists and is not an empty directory.


# import 

In [2]:
from c620.F import F
from c620.config import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.
From https://github.com/skywalker0803r/c620
   0fc2a47..eb5835c  main       -> origin/main
Updating 0fc2a47..eb5835c
Fast-forward
 F.py                           |   10 +-
 config.py                      |    2 +
 model/c660_op_real_data.pkl    |  Bin 0 -> 142789 bytes
 model/c670_op_real_data.pkl    |  Bin 0 -> 256790 bytes
 notebook/linear_op_model.ipynb | 1306 ++++++++++++++++++++++++++++++++++++----
 usage_example(colab).ipynb     |  346 ++++++++++-
 6 files changed, 1536 insertions(+), 128 deletions(-)
 create mode 100644 model/c660_op_real_data.pkl
 create mode 100644 model/c670_op_real_data.pkl


# functions

In [3]:
def check_two_data(d1,d2):
  if np.allclose(d1.values,d2.values):
    print('一樣')
  else:
    print('不一樣')

# demo data

In [4]:
tag = 'test001'
demo = joblib.load('./data/demo(real_data).pkl')
print(demo.keys())

icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']

icg_input

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c660_op', 'c670_op', 'c620_wt', 'c660_wt', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
2020-12-02,175.679504,2.0360,12.014,37.375,19.209358,2.6540,66.758,20.433,93.699834,0.6000,44.100,42.370,86.253,750.0,0.00000,28.0,39.799891
2020-10-27,180.119734,1.7560,12.078,37.252,16.990028,2.6470,72.345,18.801,89.544441,0.5456,45.773,41.738,84.166,641.0,0.00000,9.0,40.353683
2020-12-09,175.768234,1.9500,11.982,37.452,17.881016,4.3650,61.243,25.683,99.247704,0.5820,44.515,41.184,89.366,725.0,0.00000,25.0,42.253014
2020-11-11,178.663300,2.1171,11.713,37.016,17.128589,3.0507,64.741,23.511,97.068145,0.6100,45.780,42.270,85.363,802.0,0.00000,41.0,41.447528
2020-12-26,175.563812,1.9300,12.121,37.388,19.578745,2.9330,67.392,20.235,99.517662,0.6661,45.687,40.464,87.840,805.0,0.00000,52.0,41.042450
2020-11-13,177.578588,1.9270,11.817,37.221,17.044234,2.9440,65.069,22.560,98.546633,0.5700,45.070,42.390,85.351,766.0,0.00000,27.0,43.202930
2021-02-12,184.623123,1.6500,11.620,37.290,17.834211,2.4700,68.250,21.330,95.943756,0.6650,43.490,42.228,85.840,877.0,0.00000,34.0,41.703621
2021-02-13,182.727005,2.0600,11.670,37.270,17.901134,2.3700,68.700,21.000,98.045372,0.6650,43.532,42.478,87.140,838.0,0.00000,29.0,40.842743
2020-11-06,165.396682,1.3300,12.360,38.000,19.375314,3.0400,66.910,19.300,100.839962,0.6410,46.095,39.667,84.620,788.0,0.00000,38.0,43.047926
2020-12-22,182.927429,2.3960,12.175,37.270,18.905632,2.8660,67.411,20.359,97.379578,0.6700,43.810,42.410,86.005,809.0,0.00000,50.0,38.472740


In [5]:
idx1 = icg_input.index[0]
idx2 = icg_input.index[1]

# 1.確保給定不同的輸入 會有不同的輸出

In [6]:
f = F(config)
f.Recommended_mode = True #(True or False)
f.real_data_mode = True
f._Post_processing = True

# 輸入1
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

# 輸入2
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2 = f(icg_input.loc[[idx2]],c620_feed.loc[[idx2]],t651_feed.loc[[idx2]])

current Distillate Rate_m3/hr:0.22340500564133225 NA in Benzene_ppmw:980.0
current Distillate Rate_m3/hr:0.3457234686312794 NA in Benzene_ppmw:980.0


In [7]:
check_two_data(c620_wt1,c620_wt2)
check_two_data(c620_op1,c620_op2)
check_two_data(c660_wt1,c660_wt2)
check_two_data(c660_op1,c660_op2)
check_two_data(c670_wt1,c670_wt2)
check_two_data(c670_op1,c670_op2)

不一樣
不一樣
不一樣
不一樣
不一樣
不一樣


# 2.確保變更模式 推薦 和 試算 模式 有不同的輸出

In [8]:
# 試算模式
f.Recommended_mode = False #(True or False)
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

# 推薦模式
f.Recommended_mode = True #(True or False)
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

current Distillate Rate_m3/hr:0.22340500564133225 NA in Benzene_ppmw:980.0


In [9]:
check_two_data(c620_wt1,c620_wt2)
check_two_data(c620_op1,c620_op2)
check_two_data(c660_wt1,c660_wt2)
check_two_data(c660_op1,c660_op2)
check_two_data(c670_wt1,c670_wt2)
check_two_data(c670_op1,c670_op2)

不一樣
不一樣
不一樣
不一樣
不一樣
不一樣


# 確認icg_input 對應到後續系統的輸出值一致 主要有兩項要一致
* 1.benzene in c620 side
* 2.na in benzene

In [10]:
# 真實數據 試算模式
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = False #(True or False)
f.real_data_mode = True
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

            Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
2020-12-02                                             86.253               
            Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
2020-12-02                                              750.0     
            Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
2020-12-02                                             86.253                                   


2020-12-02    750.0
dtype: float64

In [11]:
# 真實數據 推薦模式
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = True #(True or False)
f.real_data_mode = True
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

            Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
2020-12-02                                             86.253               
            Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
2020-12-02                                              750.0     
current Distillate Rate_m3/hr:0.22340500564133225 NA in Benzene_ppmw:980.0
            Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
2020-12-02                                               70.0                                   


2020-12-02    980.0
dtype: float64

In [12]:
# 模擬數據 試算模式
demo = joblib.load('./data/demo.pkl')
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
idx1 = icg_input.index[0]
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = False #(True or False)
f.real_data_mode = False
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

         Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
096-002                                               90.0               
         Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
096-002                                              860.0     
         Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
096-002                                               90.0                                   


096-002    860.0
dtype: float64

In [13]:
# 模擬數據 推薦模式
demo = joblib.load('./data/demo.pkl')
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
idx1 = icg_input.index[0]
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = True #(True or False)
f.real_data_mode = False
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

         Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
096-002                                               90.0               
         Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
096-002                                              860.0     
current Distillate Rate_m3/hr:0.5736505153872938 NA in Benzene_ppmw:980.0
         Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
096-002                                               70.0                                   


096-002    980.0
dtype: float64

# 操作條件預測太準的問題

In [47]:
demo = joblib.load('./data/demo(real_data).pkl')
print(demo.keys())
idx = demo['icg_input'].sample(1).index[0]
f.Recommended_mode = True #(True or False)
f.real_data_mode = True
f._linear_op_model = False
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f(demo['icg_input'].loc[[idx]],demo['c620_feed'].loc[[idx]],demo['t651_feed'].loc[[idx]])

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c660_op', 'c670_op', 'c620_wt', 'c660_wt', 'c670_wt'])
current Distillate Rate_m3/hr:0.2937528180580594 NA in Benzene_ppmw:980.0


In [48]:
(c620_op.iloc[:,-2:] - demo['c620_op'].loc[[idx]].iloc[:,-2:])**2

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
2020-11-28,47.57374,4.453181


In [49]:
(c660_op.iloc[:,-2:]-demo['c660_op'].loc[[idx]].iloc[:,-2:])**2

,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
2020-11-28,3.635237,3.81208


In [50]:
(c670_op.iloc[:,-2:]-demo['c670_op'].loc[[idx]].iloc[:,-2:])**2

,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
2020-11-28,0.004545,2.350853


# 把操作模組改成線性模組

In [51]:
f.Recommended_mode = True #(True or False)
f.real_data_mode = True
f._linear_op_model = True
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f(demo['icg_input'].loc[[idx]],demo['c620_feed'].loc[[idx]],demo['t651_feed'].loc[[idx]])

current Distillate Rate_m3/hr:0.2937528180580594 NA in Benzene_ppmw:980.0


In [52]:
(c620_op.iloc[:,-2:] - demo['c620_op'].loc[[idx]].iloc[:,-2:])**2

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
2020-11-28,9.624155,560.194233


In [53]:
(c660_op.iloc[:,-2:]-demo['c660_op'].loc[[idx]].iloc[:,-2:])**2

,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
2020-11-28,3.635237,3.81208


In [54]:
(c670_op.iloc[:,-2:]-demo['c670_op'].loc[[idx]].iloc[:,-2:])**2

,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
2020-11-28,0.004545,2.350853
